In [2]:
import typing

import pandas as pd

import config
import src

In [3]:
CFG = config.Config()

In [3]:
dataset: pd.DataFrame = pd.read_csv('data/publicsphere/full_data.csv')
dataset

,StartDate,RecordedDate,IPAddress,Finished,Coder,ID,Mark_ID,Genre,topiccode,Platform,...,dislikeCount_video,likeCount_video,date_difference,commentCount_video,replyCount_comment,topic,subscribers,HATELIST_FOCUSED_DUMMY,Time_comment_year,Time_video_year
0,5/30/2021 13:03:17,5/30/2021 13:04:17,62.194.51.29,1,6,UgyPHwv8G0cDE6-wEgl4AaABAg.8_0ZjJKSJty8_0kXGkAd2U,119,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2017,2017.0
1,10/11/2021 10:34:05,10/11/2021 10:36:46,213.127.109.191,1,6,Ugx2WXq9UdV8mPPjejJ4AaABAg.8yHCKV0Boe58yYRxEQEF45,282,1,2,1,...,195.0,3817.0,743.0,1748.0,NaN,economy,3630000.0,0,2019,2019.0
2,9/9/2021 18:49:48,9/9/2021 18:51:32,213.127.110.0,1,6,1110578710648890000,372,2,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3,6/6/2021 16:12:46,6/6/2021 16:16:16,213.127.76.145,1,6,UgwUPFScjJ0MCeaP2F54AaABAg.8lvp3fc9Euf8lvvgsUgEgV,769,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0
4,6/13/2021 13:25:49,6/13/2021 13:27:28,213.127.82.232,1,6,UgwWKCWtSJdFvjGHvTp4AaABAg.8kUC5dGrQ2H8kUDRihE2f3,1206,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2018,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,8/19/2021 14:50:13,8/19/2021 14:54:28,62.194.51.29,1,6,1152219467579100000,10000695,0,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3858,8/19/2021 15:10:27,8/19/2021 15:12:21,62.194.51.29,1,6,1085362296472430000,10007008,1,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2019,NaN
3859,10/6/2021 16:08:39,10/6/2021 16:10:42,213.127.113.113,1,6,UghFY3QJ6nmT_ngCoAEC.7-H0Z7--wxd8goqpaPs-bl,20000102,0,3,1,...,2820.0,12475.0,3803.0,4785.0,NaN,east,6740000.0,0,2018,2010.0
3860,10/15/2021 18:30:04,10/15/2021 18:35:40,213.127.109.191,1,6,UgyWabsmmnq3zam4DgZ4AaABAg,20000418,2,3,1,...,118.0,31761.0,1531.0,2206.0,0.0,east,6800000.0,0,2018,2015.0


In [4]:
MODEL: str = "llama3:70b-instruct-q6_K"
CFG.prompt_classify_files.keys()

dict_keys(['interactivity_acknowledgement', 'political_ideology', 'rationality_external_evidence', 'rationality_topic_relevance', 'political_negativity', 'rationality_background_info', 'rationality_reasoning'])

In [11]:
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["commentText"], MODEL)
    )
    for label, path in CFG.prompt_classify_files.items()
}

classifying rationality_reasoning: 100%|██████████| 3862/3862 [52:02<00:00,  1.24it/s]  


In [ ]:
predictions

In [12]:
dataset_w_pred = dataset.copy()

for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    dataset_w_pred = dataset_w_pred.join(preds)
    

No     2526
Yes    1332
Name: interactivity_acknowledgement, dtype: int64
------------------------------------------
neutral         1750
liberal         1325
conservative     784
Name: political_ideology, dtype: int64
------------------------------------------
No     3251
Yes     600
Name: rationality_external_evidence, dtype: int64
------------------------------------------
Yes    2555
No     1303
Name: rationality_topic_relevance, dtype: int64
------------------------------------------
political/negative        2560
not political/negative    1297
Name: political_negativity, dtype: int64
------------------------------------------
No     3158
Yes     700
Name: rationality_background_info, dtype: int64
------------------------------------------
No     2280
Yes    1576
Name: rationality_reasoning, dtype: int64
------------------------------------------


In [ ]:
dataset_w_pred.to_json(f'{CFG.report_dir}/publicsphere.prompt_classify.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred.to_parquet(f'{CFG.report_dir}/publicsphere.prompt_classify.parquet')
dataset_w_pred

In [5]:
MODELS: typing.Tuple[str, str] = [
    ("topics", "cardiffnlp/tweet-topic-21-multi"),
    ("emotions", "cardiffnlp/twitter-roberta-base-emotion-multilabel-latest"),
    ("sentiment", "cardiffnlp/twitter-roberta-base-sentiment-latest"),
    ("irony", "cardiffnlp/twitter-roberta-base-irony"),
    ("offensive",  "cardiffnlp/twitter-roberta-base-offensive"),
    ("hate", "cardiffnlp/twitter-roberta-base-hate-latest")
]

In [4]:
Cardiff_predictions: typing.Dict[str, pd.Series] = {}

In [ ]:
for label, model in MODELS:
    classifier = src.HFClassify(slug=model)
    Cardiff_predictions[label] = pd.Series(classifier(dataset["commentText"], theta=0.4))

In [7]:
Cardiff_predictions

{'offensive': 0       {offensive, non-offensive}
 1       {offensive, non-offensive}
 2                  {non-offensive}
 3                  {non-offensive}
 4                  {non-offensive}
                    ...            
 3857               {non-offensive}
 3858               {non-offensive}
 3859               {non-offensive}
 3860               {non-offensive}
 3861               {non-offensive}
 Length: 3862, dtype: object}

In [8]:
for label, series in Cardiff_predictions.items():
    display(label)
    display(series.value_counts())

'offensive'

{non-offensive}               2066
{offensive, non-offensive}    1158
{offensive}                    638
Name: count, dtype: int64

In [ ]:
dataset_w_pred = pd.read_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify.parquet')

for labels, preds in Cardiff_predictions.items():
    dataset_w_pred_2 = dataset_w_pred.join(preds.rename(labels))

dataset_w_pred_2

In [24]:
dataset_w_pred_2.loc[:, 'offensive'] = dataset_w_pred_2.loc[:, 'offensive'].astype('str').astype('category')

In [26]:
dataset_w_pred_2.to_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_2.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_2.to_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_2.parquet')

In [7]:
#anonymize:
dataset_w_pred_anon = pd.read_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_s.parquet')
dataset_w_pred_anon = dataset_w_pred_anon.drop('commentText', axis=1)


In [8]:
dataset_w_pred_anon.to_json(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_anon.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred_anon.to_parquet(f'{CFG.report_dir}/publicsphere.cardiff_prompt_classify_anon.parquet')